# Combining ACCEPTED loans data and REJECTED loans data

- This notebook contains code of merging datasets in order to try and predict wheter loan is accepted or rejected. 
- There were two separate datasets exclusively made up of either accepted or rejected loans.
- Task here was to identify any potential connections between them to merge based on.
- Task wasnt perfect but I've found some overlaping features. 
- If the primary business goal is to predict this acceptance or rejection, business has to collect more simmilar features on both types of loans. 
- Imbalanced datasets for classification with 10x difference between classification categories. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier 

In [18]:
accepted_path = './Datasets/accepted_2007_to_2018Q4.csv'
rejected_path = './Datasets/rejected_2007_to_2018Q4.csv'

In [19]:
data_accepted = pd.read_csv(accepted_path)
data_rejected = pd.read_csv(rejected_path)

C:\Users\Martin\AppData\Local\Temp\ipykernel_16200\618601729.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  data_accepted = pd.read_csv(accepted_path)


## Shape of the data

In [20]:
# Displaying the shape of the datasets
print("Accepted Loans:", data_accepted.shape)
print("Rejected Loans:", data_rejected.shape)

Accepted Loans: (2260701, 151)
Rejected Loans: (27648741, 9)


- We see rejected has 10x the number of instances, with more than 10x less features

In [21]:
# Exploring the column names
print("Columns in Accepted Loans:", data_accepted.columns.values)

Columns in Accepted Loans: ['id' 'member_id' 'loan_amnt' 'funded_amnt' 'funded_amnt_inv' 'term'
 'int_rate' 'installment' 'grade' 'sub_grade' 'emp_title' 'emp_length'
 'home_ownership' 'annual_inc' 'verification_status' 'issue_d'
 'loan_status' 'pymnt_plan' 'url' 'desc' 'purpose' 'title' 'zip_code'
 'addr_state' 'dti' 'delinq_2yrs' 'earliest_cr_line' 'fico_range_low'
 'fico_range_high' 'inq_last_6mths' 'mths_since_last_delinq'
 'mths_since_last_record' 'open_acc' 'pub_rec' 'revol_bal' 'revol_util'
 'total_acc' 'initial_list_status' 'out_prncp' 'out_prncp_inv'
 'total_pymnt' 'total_pymnt_inv' 'total_rec_prncp' 'total_rec_int'
 'total_rec_late_fee' 'recoveries' 'collection_recovery_fee'
 'last_pymnt_d' 'last_pymnt_amnt' 'next_pymnt_d' 'last_credit_pull_d'
 'last_fico_range_high' 'last_fico_range_low' 'collections_12_mths_ex_med'
 'mths_since_last_major_derog' 'policy_code' 'application_type'
 'annual_inc_joint' 'dti_joint' 'verification_status_joint'
 'acc_now_delinq' 'tot_coll_amt' 'tot

In [22]:
print("Columns in Rejected Loans:", data_rejected.columns.values)

Columns in Rejected Loans: ['Amount Requested' 'Application Date' 'Loan Title' 'Risk_Score'
 'Debt-To-Income Ratio' 'Zip Code' 'State' 'Employment Length'
 'Policy Code']


In [23]:
# Displaying the first few rows of each dataset
data_accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
data_rejected.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


- Column Name Formatting: Inconsistencies noted, standardization needed for easier data integration.
- Data Collection Optimization: Structuring loan data collection to enhance prediction of loan acceptance/rejection.
- Consistent Feature Set: Essential to have the same features in both accepted and rejected loan datasets for effective modeling.
- Automated Decision-Making: Crucial to collect data in a uniform manner, differing only in loan status (accepted or rejected).
- Exploring Commonalities: Investigating similarities between datasets to facilitate merging and modeling.

## Common features and their reformating

In [25]:
common_features = ['loan_amnt', 'title', 'dti', 'zip_code', 'addr_state', 'emp_length']

In [26]:
# Extracting overlapping features and 'issue_d' from the accepted dataset
data_accepted_overlap = data_accepted[common_features + ['issue_d']]

In [27]:
# Converting DTI from % to floats in rejected dataset
data_rejected['Debt-To-Income Ratio'] = data_rejected['Debt-To-Income Ratio'].str.rstrip('%').astype('float')

In [28]:
# Renaming columns for consistency
rename_columns = {
    'Debt-To-Income Ratio': 'dti',
    'Employment Length': 'emp_length',
    'Application Date': 'year',
    'Amount Requested': 'loan_amnt',
    'Loan Title': 'title',
    'State': 'addr_state',
    'Zip Code': 'zip_code',
    'Policy Code': 'policy_code'
}
data_rejected.rename(columns=rename_columns, inplace=True)

In [29]:
# Extracting the year from dates in order to have time dimension
data_rejected['year'] = pd.to_datetime(data_rejected['year']).dt.year
data_accepted_overlap['issue_d'] = pd.to_datetime(data_accepted_overlap['issue_d'], format='%b-%Y').dt.year
data_accepted_overlap.rename(columns={'issue_d': 'year'}, inplace=True)

C:\Users\Martin\AppData\Local\Temp\ipykernel_16200\2613273085.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_accepted_overlap['issue_d'] = pd.to_datetime(data_accepted_overlap['issue_d'], format='%b-%Y').dt.year
C:\Users\Martin\AppData\Local\Temp\ipykernel_16200\2613273085.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_accepted_overlap.rename(columns={'issue_d': 'year'}, inplace=True)


In [30]:
# Dropping the policy code
data_rejected.drop(['policy_code','Risk_Score'], axis=1, inplace=True)

In [31]:
# Adding loan status column
data_accepted_overlap['loan_status'] = 1  # 1 for accepted
data_rejected['loan_status'] = 0  # 0 for rejected

C:\Users\Martin\AppData\Local\Temp\ipykernel_16200\162931315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_accepted_overlap['loan_status'] = 1  # 1 for accepted


In [32]:
data_accepted_overlap.loan_status.value_counts()

loan_status
1    2260701
Name: count, dtype: int64

In [33]:
data_rejected['loan_status'].value_counts()

loan_status
0    27648741
Name: count, dtype: int64

## Merging the datasets

In [34]:
# Merging the datasets
combined_df = pd.concat([data_accepted_overlap, data_rejected], ignore_index=True)

In [35]:
combined_df.head()

,loan_amnt,title,dti,zip_code,addr_state,emp_length,year,loan_status
0,3600.0,Debt consolidation,5.91,190xx,PA,10+ years,2015.0,1
1,24700.0,Business,16.06,577xx,SD,10+ years,2015.0,1
2,20000.0,NaN,10.78,605xx,IL,10+ years,2015.0,1
3,35000.0,Debt consolidation,17.06,076xx,NJ,10+ years,2015.0,1
4,10400.0,Major purchase,25.37,174xx,PA,3 years,2015.0,1


## Modifying 'title' format and re-categorizing the feature 

In [36]:
# Cleaning 'title' column, standardizing it's format
combined_df['title'] = combined_df['title'].str.lower().str.replace('_', ' ').str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [37]:
# Categorizing 'title' using predefined mapping
def categorize_title(title):
    if pd.isna(title):
        return pd.NA
    for category, keywords in title_mapping.items():
        if any(keyword in title for keyword in keywords):
            return category
    return pd.NA

title_mapping = {
    'Debt Consolidation': ['debt consolidation', 'consolidation', 'consolidate', 'payoff'],
    'Credit Card': ['credit card'],
    'Home Related': ['home', 'house'],
    'Car Related': ['car'],
    'Business': ['business'],
    'Medical Expenses': ['medical'],
    'Moving and Relocation': ['moving'],
    'Vacation': ['vacation'],
    'Major Purchase': ['major purchase'],
    'Green Loan': ['renewable energy', 'green loan'],
    'Wedding': ['wedding'],
    'Personal Loan': ['personal'],
    'Educational': ['educational'],
    'Freedom': ['freedom'],
    'Loan': ['loan'],
    'Other': ['other']
}

In [38]:
combined_df['title_category'] = combined_df['title'].apply(categorize_title)

In [39]:
# Dropping the original 'title' column
combined_df.drop('title', axis=1, inplace=True)

## Final combined dataset

In [40]:
combined_df.head()

,loan_amnt,dti,zip_code,addr_state,emp_length,year,loan_status,title_category
0,3600.0,5.91,190xx,PA,10+ years,2015.0,1,Debt Consolidation
1,24700.0,16.06,577xx,SD,10+ years,2015.0,1,Business
2,20000.0,10.78,605xx,IL,10+ years,2015.0,1,<NA>
3,35000.0,17.06,076xx,NJ,10+ years,2015.0,1,Debt Consolidation
4,10400.0,25.37,174xx,PA,3 years,2015.0,1,Major Purchase


In [41]:
combined_df.shape

(29909442, 8)

- Saving the combined dataset

In [42]:
combined_df.to_csv('./Datasets/combined_loans.csv', index=False)